<a href="https://colab.research.google.com/github/kavyajeetbora/zads/blob/master/development/03_real_estate_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json

All the coordinates data can also be listed using regex pattern:

```
\d+\.\d+, \d+\.\d+
```

after this filter out the invalid coodinates

In [52]:
HEADER = {
      'User-Agent': 'Mozilla/5.0 (X11; CrOS x86_64 14541.0.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
      'Cookie': "place_your_cookie_here",
      'Referer': 'https://www.google.com/',
      'Accept' : '*/*',
      'Accept-Language' : 'en-US,en;q=0.9'
}

URL = "https://www.google.com/search"


def geocoder(place_name):

    PARAMS = {
        'tbm': 'map',
        'authuser': '0',
        'hl': 'en',
        'q': place_name,
        'tch': '1',
        'ech': '5'
    }

    response = requests.get(URL, params=PARAMS, headers=HEADER)
    messy_data = response.text

    data = json.loads(messy_data[:-6])
    javascript_array_str = data["d"][5:]
    python_list = json.loads(javascript_array_str)
    places_data = python_list[0][1]

    _, lon, lat = python_list[1][0]
    return lat, lon

In [61]:
place_name = "Pareena Micasa, Sector 68, Gurugram, Haryana 122101"
geocoder(place_name)

(28.388229299999995, 77.0420637)

## Scraping from 99 acres


In [62]:
!pip install -q fake_useragent

In [67]:
import random
from fake_useragent import UserAgent
from lxml import html, etree

In [65]:
def get_random_user_agent():
    try:
        ua = UserAgent()
        return ua.random
    except:
        # Fallback user agents if fake_useragent fails
        user_agents = [
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/120.0.0.0 Safari/537.36',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2 Safari/605.1.15'
        ]
        return random.choice(user_agents)

def make_request(url):
    # Headers to mimic a browser
    headers = {
        'User-Agent': get_random_user_agent(),
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Cache-Control': 'max-age=0',
        'DNT': '1',  # Do Not Track
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
    }

    # Parameters for the request
    params = {
        'city': '8',
        # Add any additional parameters needed
        'preference': 'S',  # S for Sale
        'area_unit': '1',   # Square Feet
        'res_com': 'R',     # Residential
    }

    try:
        # Make the request with a timeout
        response = requests.get(
            url,
            headers=headers,
            params=params,
            timeout=30,
            verify=True  # SSL verification
        )

        # Raise an exception for bad status codes
        response.raise_for_status()

        return response

    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
        return None

In [70]:
city_url = 'https://www.99acres.com/search/property/buy/'
response = make_request(city_url)

if response and response.status_code == 200:
    print("Request successful!")
else:
    print("Request failed!")

Request successful!


In [71]:
# Create etree from response text
tree = html.fromstring(response.content)

# Make links absolute (optional but recommended)
tree.make_links_absolute(response.url)

In [88]:
cards = tree.xpath('''//*[@class="ellipsis"]''')
for card in cards:
    print(card.text)

Ganga Anantam
M3M Mansion
M3M Antalya Hills
Smartworld One DXP
Puri Diplomatic Residences
Godrej Zenith
Signature Global Titanium SPR
Elan The Presidential
M3M Altitude
Godrej Aristocrat
Silverglades The Legacy
Whiteland The Aspen
Krisumi Waterside Residences
Smartworld The Edition
TARC Ishva
Pyramid Alban
M3M Golf Hills
M3M Crown
4S The Aurrum
Ganga Nandaka
Signature Global Daxin Vistas
Oxirich Chintamanis
M3M Capital
Adani Lushlands
Sobha Aranya


In [69]:
from lxml import html, etree
import requests
from urllib.parse import urljoin
import re

def create_etree(response):
    """
    Creates an lxml etree from response text and handles common errors
    """
    try:
        # Create etree from response text
        tree = html.fromstring(response.content)

        # Make links absolute (optional but recommended)
        tree.make_links_absolute(response.url)

        return tree

    except etree.ParserError as e:
        print(f"Parser error: {e}")
        # Try cleaning the HTML first
        cleaned_html = clean_html(response.text)
        return html.fromstring(cleaned_html)

    except Exception as e:
        print(f"Error creating etree: {e}")
        return None

def clean_html(html_text):
    """
    Cleans problematic HTML content
    """
    # Remove null bytes
    html_text = html_text.replace('\x00', '')

    # Fix common HTML issues
    html_text = re.sub(r'<\s+', '<', html_text)  # Fix tags with extra spaces
    html_text = re.sub(r'\s+>', '>', html_text)

    # Fix unclosed tags (basic)
    common_tags = ['div', 'span', 'p', 'a', 'li', 'ul', 'table', 'tr', 'td']
    for tag in common_tags:
        pattern = f'<{tag}(?![^>]*/>)[^>]*>'
        closing_pattern = f'</{tag}>'
        opens = len(re.findall(pattern, html_text, re.IGNORECASE))
        closes = len(re.findall(closing_pattern, html_text, re.IGNORECASE))
        # Add missing closing tags
        if opens > closes:
            html_text += f'</{tag}>' * (opens - closes)

    return html_text

def parse_99acres(response):
    """
    Parse 99acres.com property listings
    """
    if not response or response.status_code != 200:
        print("Invalid response")
        return None

    # Create etree
    tree = create_etree(response)
    if tree is None:
        return None

    try:
        # Example parsing (adjust XPaths based on actual HTML structure)
        properties = []

        # Find all property listings
        listing_xpath = "//div[contains(@class, 'property-listing')]"  # Adjust this
        listings = tree.xpath(listing_xpath)

        for listing in listings:
            try:
                property_data = {
                    'title': get_text(listing, ".//h2"),
                    'price': get_text(listing, ".//div[contains(@class, 'price')]"),
                    'location': get_text(listing, ".//div[contains(@class, 'location')]"),
                    'area': get_text(listing, ".//div[contains(@class, 'area')]"),
                    'link': get_attr(listing, ".//a[@class='property-link']", 'href'),
                }
                properties.append(property_data)
            except Exception as e:
                print(f"Error parsing listing: {e}")
                continue

        return properties

    except Exception as e:
        print(f"Error parsing page: {e}")
        return None

def get_text(element, xpath, default=''):
    """
    Safely extract text from an xpath
    """
    try:
        result = element.xpath(xpath)
        if result:
            return ' '.join(result[0].text_content().split())
        return default
    except:
        return default

def get_attr(element, xpath, attr, default=''):
    """
    Safely extract attribute from an xpath
    """
    try:
        result = element.xpath(xpath)
        if result:
            return result[0].get(attr, default)
        return default
    except:
        return default


city_url = 'https://www.99acres.com/search/property/buy/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
}
params = {'city': '8'}

try:
    response = requests.get(city_url, headers=headers, params=params, timeout=30)
    properties = parse_99acres(response)

    if properties:
        print(f"Found {len(properties)} properties")
        for prop in properties:
            print("\nProperty Details:")
            for key, value in prop.items():
                print(f"{key}: {value}")
    else:
        print("No properties found or error occurred")

except Exception as e:
    print(f"Error in main: {e}")

Error in main: HTTPSConnectionPool(host='www.99acres.com', port=443): Read timed out. (read timeout=30)
